# Training Playground

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from messenger_dataset import MessengerDataset

In [2]:
LEARNING_RATE = 1e-3
BATCH_SIZE = 64
EPOCHS = 5

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

Device: cpu


In [4]:
ds = MessengerDataset("merged/df_train.csv", window_size=7)
dl = DataLoader(ds, batch_size=BATCH_SIZE)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten(-2, -1)
        self.linear1 = nn.Linear(9, 7)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(7, 5)

    def forward(self, x):
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)
        return x


net = Net()
net.to(device)
net.double()  # TODO maybe rather convert input to float
print(net)

Net(
  (flatten): Flatten()
  (linear1): Linear(in_features=9, out_features=7, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=7, out_features=5, bias=True)
)


In [6]:
sample, label = ds[32246]

print(sample)
print(label)

tensor([[[-0.5450,  0.6900, -0.3070],
         [ 0.1106,  0.1128, -0.4642],
         [-0.9935, -0.3451,  0.0630]],

        [[-0.5890,  0.6708, -0.3210],
         [ 0.1106,  0.1128, -0.4643],
         [-0.9935, -0.3451,  0.0630]],

        [[-0.5825,  0.6750, -0.3174],
         [ 0.1107,  0.1129, -0.4643],
         [-0.9935, -0.3451,  0.0630]],

        [[-0.5555,  0.6845, -0.2923],
         [ 0.1107,  0.1130, -0.4644],
         [-0.9935, -0.3451,  0.0630]],

        [[-0.5844,  0.7142, -0.3132],
         [ 0.1108,  0.1130, -0.4645],
         [-0.9935, -0.3451,  0.0630]],

        [[-0.5863,  0.7297, -0.4487],
         [ 0.1108,  0.1131, -0.4645],
         [-0.9935, -0.3451,  0.0630]],

        [[-0.5598,  0.7488, -0.4011],
         [ 0.1109,  0.1131, -0.4646],
         [-0.9935, -0.3451,  0.0630]]], dtype=torch.float64)
tensor([2, 2, 2, 2, 1, 1, 1])


In [7]:
out = net(sample)
print(out)
print(out.shape)

tensor([[-0.3664, -0.0523, -0.4558, -0.2804,  0.1764],
        [-0.3694, -0.0570, -0.4537, -0.2828,  0.1771],
        [-0.3689, -0.0561, -0.4541, -0.2824,  0.1768],
        [-0.3640, -0.0515, -0.4546, -0.2783,  0.1736],
        [-0.3735, -0.0539, -0.4579, -0.2861,  0.1773],
        [-0.3982, -0.0674, -0.4641, -0.3071,  0.1996],
        [-0.3908, -0.0602, -0.4648, -0.3009,  0.1931]], dtype=torch.float64,
       grad_fn=<AddmmBackward>)
torch.Size([7, 5])


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)

In [16]:
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}\n-------------------------------")
    size = len(dl.dataset)
    for batch, (X, y) in enumerate(dl):
        pred = net(X)
        print(pred.shape)
        pred = pred.permute(0, 2, 1)
        loss = criterion(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>10d}/{size:>5d}]")

print("DONE.")

Epoch 1
-------------------------------
torch.Size([64, 7, 5])
loss: 0.003105  [         0/7493622]


KeyboardInterrupt: 

In [11]:
pred = torch.tensor([[0., 1], [1, 0]])
target = torch.tensor([1, 0])
print(pred)
print(target)
criterion(pred, target)

tensor([[0., 1.],
        [1., 0.]])
tensor([1, 0])


tensor(0.3133)